# Homework 3

## FINM 35700 - Spring 2024

### UChicago Financial Mathematics

### Due Date: 2024-04-23

* Alex Popovici
* alex.popovici@uchicago.edu

This homework relies on:

- the SOFR OIS symbology file `sofr_swap_symbology`,
- the SOFR swaps market data file `sofr_swaps_market_data_eod` and
- the CDS spreads market data file `cds_market_data_eod`.

In [104]:
import pandas as pd
import scipy as sp
import QuantLib as ql
import numpy as np
import pandas as pd
import datetime as dt

cds_market_data_file = pd.read_csv('/Users/rogerlin/Downloads/UChicago_FINM_35700_CreditMarkets_Spring2024_Homework3/data/cds_market_data_eod.csv')
sofr_swap_symbology_file = pd.read_csv('/Users/rogerlin/Downloads/UChicago_FINM_35700_CreditMarkets_Spring2024_Homework3/data/sofr_swaps_symbology.csv')
sofr_swap_market_data_file = pd.read_csv('/Users/rogerlin/Downloads/UChicago_FINM_35700_CreditMarkets_Spring2024_Homework3/data/sofr_swaps_market_data_eod.csv')
print(cds_market_data_file.to_string()) 
print(sofr_swap_symbology_file.to_string()) 
print(sofr_swap_market_data_file.to_string()) 



       date ticker                short_name    tier      sector  region currency doc_clause  running_coupon  cds_assumed_recovery  par_spread_1y  par_spread_2y  par_spread_3y  par_spread_5y  par_spread_7y  par_spread_10y
0    1/2/24    IBM  Intl Business Machs Corp  SNRFOR  Technology  N.Amer      USD       XR14            0.01                   0.4        13.6831        18.8194        28.3917        44.7053        62.1494         69.1972
1    1/3/24    IBM  Intl Business Machs Corp  SNRFOR  Technology  N.Amer      USD       XR14            0.01                   0.4        14.2256        19.6610        29.4493        46.4866        63.6475         71.4311
2    1/4/24    IBM  Intl Business Machs Corp  SNRFOR  Technology  N.Amer      USD       XR14            0.01                   0.4        13.8318        19.1828        28.8454        45.4735        62.6543         70.9180
3    1/5/24    IBM  Intl Business Machs Corp  SNRFOR  Technology  N.Amer      USD       XR14            0.01    

In [105]:
import QuantLib as ql
import numpy as np
import pandas as pd
import datetime as dt

def get_ql_date(date) -> ql.Date:
    """
    convert dt.date to ql.Date
    """
    if isinstance(date, dt.date):
        return ql.Date(date.day, date.month, date.year)
    elif isinstance(date, str):
        date = dt.datetime.strptime(date, "%Y-%m-%d").date()
        return ql.Date(date.day, date.month, date.year)
    else:
        raise ValueError(f"to_qldate, {type(date)}, {date}")
    
def create_schedule_from_symbology(details: dict):
    '''Create a QuantLib cashflow schedule from symbology details dictionary (usually one row of the symbology dataframe)
    '''
    
    # Create maturity from details['maturity']
    maturity = get_ql_date(details['maturity'])
    
    # Create acc_first from details['acc_first']
    acc_first =  get_ql_date(details['acc_first'])
    
    # Create calendar for Corp and Govt asset classes
    calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
    
    # define period from details['cpn_freq'] ... can be hard-coded to 2 = semi-annual frequency
    period = ql.Period(2)
    
    # business_day_convention
    business_day_convention = ql.Unadjusted
    
    # termination_date_convention
    termination_date_convention = ql.Unadjusted
    
    # date_generation
    date_generation=ql.DateGeneration.Backward
    
    # Create schedule using ql.MakeSchedule interface (with keyword arguments)
    schedule = ql.MakeSchedule(effectiveDate=acc_first,  # this may not be the same as the bond's start date
                            terminationDate=maturity,
                            tenor=period,
                            calendar=calendar,
                            convention=business_day_convention,
                            terminalDateConvention=termination_date_convention,
                            rule=date_generation,
                            endOfMonth=True,
                            firstDate=ql.Date(),
                            nextToLastDate=ql.Date())
    return schedule

def create_bond_from_symbology(details: dict):
    '''Create a US fixed rate bond object from symbology details dictionary (usually one row of the symbology dataframe)
    '''
    
     # Create day_count from details['dcc']
     # For US Treasuries use ql.ActualActual(ql.ActualActual.ISMA)
     # For US Corporate bonds use ql.Thirty360(ql.Thirty360.USA)
    
    if details['class'] == 'Corp':
        day_count = ql.Thirty360(ql.Thirty360.USA)
    elif details['class'] == 'Govt':
        day_count = ql.ActualActual(ql.ActualActual.ISMA)
    else:
        raise ValueError(f"unsupported asset class, {type(details['class'])}, {details['class']}")

    
    # Create issue_date from details['start_date']
    issue_date = get_ql_date(details['start_date'])
    
    # Create days_settle from details['days_settle']
    days_settle = int(float(details['days_settle']))

    # Create coupon from details['coupon']
    coupon = float(details['coupon'])/100.


    # Create cashflow schedule
    schedule = create_schedule_from_symbology(details)
    
    face_value = 100
    redemption = 100
    
    payment_convention = ql.Unadjusted
        
    # Create fixed rate bond object
    fixed_rate_bond = ql.FixedRateBond(
        days_settle,
        face_value,
        schedule,
        [coupon],
        day_count,
        payment_convention,
        redemption,
        issue_date)        

    return fixed_rate_bond

def get_bond_cashflows(bond: ql.FixedRateBond, calc_date=ql.Date) -> pd.DataFrame:
    '''Returns all future cashflows as of calc_date, i.e. with payment dates > calc_date.
    '''    
    day_counter = bond.dayCounter()    
    
    x = [(cf.date(), day_counter.yearFraction(calc_date, cf.date()), cf.amount()) for cf in bond.cashflows()]
    cf_date, cf_yearFrac, cf_amount = zip(*x)
    cashflows_df = pd.DataFrame(data={'CashFlowDate': cf_date, 'CashFlowYearFrac': cf_yearFrac, 'CashFlowAmount': cf_amount})

    # filter for payment dates > calc_date
    cashflows_df = cashflows_df[cashflows_df.CashFlowYearFrac > 0]
    return cashflows_df


def calibrate_yield_curve_from_frame(
        calc_date: ql.Date,
        treasury_details: pd.DataFrame,
        price_quote_column: str):
    '''Create a calibrated yield curve from a details dataframe which includes bid/ask/mid price quotes.
    '''
    ql.Settings.instance().evaluationDate = calc_date

    # Sort dataframe by maturity
    sorted_details_frame = treasury_details.sort_values(by='maturity')    
    
    # For US Treasuries use ql.ActualActual(ql.ActualActual.ISMA)
    day_count = ql.ActualActual(ql.ActualActual.ISMA)

    bond_helpers = []
    
    for index, row in sorted_details_frame.iterrows():
        bond_object = create_bond_from_symbology(row)
        
        tsy_clean_price_quote = row[price_quote_column]
        tsy_clean_price_handle = ql.QuoteHandle(ql.SimpleQuote(tsy_clean_price_quote))
        
        bond_helper = ql.BondHelper(tsy_clean_price_handle, bond_object)
        bond_helpers.append(bond_helper)
        
    yield_curve = ql.PiecewiseLogCubicDiscount(calc_date, bond_helpers, day_count)
    # yield_curve = ql.PiecewiseFlatForward(calc_date, bond_helpers, day_count)
    
    yield_curve.enableExtrapolation()
    return yield_curve



def get_yield_curve_details_df(yield_curve, curve_dates=None):
    
    if(curve_dates == None):
        curve_dates = yield_curve.dates()

    dates = [d.to_date() for d in curve_dates]
    discounts = [round(yield_curve.discount(d), 3) for d in curve_dates]
    yearfracs = [round(yield_curve.timeFromReference(d), 3) for d in curve_dates]
    zeroRates = [round(yield_curve.zeroRate(d, yield_curve.dayCounter(), ql.Compounded).rate() * 100, 3) for d in curve_dates]

    yield_curve_details_df = pd.DataFrame(data={'Date': dates,
                             'YearFrac': yearfracs,
                             'DiscountFactor': discounts,
                             'ZeroRate': zeroRates})                             
    return yield_curve_details_df


def calc_clean_price_with_zspread(fixed_rate_bond, yield_curve_handle, zspread):
    zspread_quote = ql.SimpleQuote(zspread)
    zspread_quote_handle = ql.QuoteHandle(zspread_quote)
    yield_curve_bumped = ql.ZeroSpreadedTermStructure(yield_curve_handle, zspread_quote_handle, ql.Compounded, ql.Semiannual)
    yield_curve_bumped_handle = ql.YieldTermStructureHandle(yield_curve_bumped)
    
    # Set Valuation engine
    bond_engine = ql.DiscountingBondEngine(yield_curve_bumped_handle)
    fixed_rate_bond.setPricingEngine(bond_engine)
    bond_clean_price = fixed_rate_bond.cleanPrice()
    return bond_clean_price


-----------------------------------------------------------
# Problem 1: Risk & Scenario analysis for a fixed rate corporate bond (yield model)
## Use the QuantLib Basic notebook (or previous homeworks) as templates.

## a. Create generic fixed-rate corporate bond
Fix the calculation date as of April 15 2024 and use a coupon of 5% and a maturity of 10 years (April 15 2034).

Display the fixed rate bond cashflows.

In [106]:
# import tools from previous homeworks
from credit_market_tools import *

# Use static calculation/valuation date of 2024-04-15, matching data available in the market prices EOD file
calc_date = ql.Date(15, 4, 2024)
ql.Settings.instance().evaluationDate = calc_date

In [107]:
# Use the bond_details template below to quickly define the bond specs
test_bond_details = {'class': 'Corp',
                'start_date': 'YYYY-MM-DD', 
                'acc_first': 'YYYY-MM-DD', 
                'maturity': 'YYYY-MM-DD', 
                'coupon': 5,
                'dcc' : '30/360',
                'days_settle' : 2}

# Use create_bond_from_symbology() to create the bond

In [108]:
# Updated bond details
test_bond_details = {
    'class': 'Corp',
    'start_date': '2024-04-15',  # Assuming bond starts on the calculation date
    'acc_first': '2024-04-15',  # Assuming accrual starts on the calculation date
    'maturity': '2034-04-15',  # 10 years maturity from the calculation date
    'coupon': 5,  # 5% coupon rate
    'dcc': '30/360',  # Day count convention
    'days_settle': 2  # Settlement days
}

# Assuming create_bond_from_symbology() and get_bond_cashflows() functions are defined as shown in the context
# Create the bond object
fixed_rate_bond = create_bond_from_symbology(test_bond_details)

# Set the calculation date
calc_date = ql.Date(15, 4, 2024)
ql.Settings.instance().evaluationDate = calc_date

# Get and display the bond cashflows
cashflows_df = get_bond_cashflows(fixed_rate_bond, calc_date)
print(cashflows_df)

          CashFlowDate  CashFlowYearFrac  CashFlowAmount
0   October 15th, 2024               0.5             2.5
1     April 15th, 2025               1.0             2.5
2   October 15th, 2025               1.5             2.5
3     April 15th, 2026               2.0             2.5
4   October 15th, 2026               2.5             2.5
5     April 15th, 2027               3.0             2.5
6   October 15th, 2027               3.5             2.5
7     April 15th, 2028               4.0             2.5
8   October 15th, 2028               4.5             2.5
9     April 15th, 2029               5.0             2.5
10  October 15th, 2029               5.5             2.5
11    April 15th, 2030               6.0             2.5
12  October 15th, 2030               6.5             2.5
13    April 15th, 2031               7.0             2.5
14  October 15th, 2031               7.5             2.5
15    April 15th, 2032               8.0             2.5
16  October 15th, 2032         

## b. Compute the bond price, DV01, duration and convexity (analytic method).

Assume that the market yield of the bond is 6%. Compute the bond price, DV01, duration and convexity, using the analytic method.

In [109]:
# Assuming fixed_rate_bond is already created and ql is imported
market_yield = 0.06  # 6%
compounding = ql.Compounded
frequency = ql.Semiannual
day_count = ql.Thirty360(ql.Thirty360.USA)

# Calculate bond price
bond_price = fixed_rate_bond.cleanPrice(market_yield, day_count, compounding, frequency)
print(f"Bond Price: {bond_price}")

# Calculate DV01
price_up = fixed_rate_bond.cleanPrice(market_yield + 0.0001, day_count, compounding, frequency)  # Yield + 1 basis point
dv01 = (price_up - bond_price) / 100  # Convert to dollar value
print(f"DV01: {dv01}")

# Calculate Modified Duration using BondFunctions
modified_duration = ql.BondFunctions.duration(fixed_rate_bond, market_yield, day_count, compounding, frequency, ql.Duration.Modified)
print(f"Modified Duration: {modified_duration}")

# Calculate Convexity using BondFunctions
convexity = ql.BondFunctions.convexity(fixed_rate_bond, market_yield, day_count, compounding, frequency)
print(f"Convexity: {convexity}")

Bond Price: 92.56388978531126
DV01: -0.0007088881432609639
Modified Duration: 7.659652218084261
Convexity: 71.70012220344685


## c. Scenario bond prices: "re-pricing" vs "second-order approximations"

Compute the scenario bond prices on the following scenario yield grid: [from 2% to 10% in steps of 0.5%]

Compute the second-order scenario price approximations using duration and convexity sensitivities.

Using Plotly, Plot the scenario prices (Y-axis) vs yieds (X-axis), for both the "re-pricing" and "second-order approximations" method.

In [110]:
import numpy as np
import plotly.graph_objects as go

# Given values
current_yield = 0.06  # Current market yield of 6%
yields = np.arange(0.02, 0.105, 0.005)  # Scenario yields from 2% to 10% in steps of 0.5%
current_price = bond_price  # Use the bond price calculated previously
modified_duration = modified_duration  # Use the modified duration calculated previously
convexity = convexity  # Use the convexity calculated previously

# Calculate scenario bond prices
scenario_prices = [fixed_rate_bond.cleanPrice(y, day_count, compounding, frequency) for y in yields]

# Calculate second-order scenario price approximations
delta_y = yields - current_yield
approx_prices = current_price * (1 - modified_duration * delta_y + 0.5 * convexity * delta_y**2)

### Step 3: Plot Using Plotly

fig = go.Figure()

# Add traces for actual re-pricing and second-order approximations
fig.add_trace(go.Scatter(x=yields*100, y=scenario_prices, mode='lines+markers', name='Re-priced Bond Prices'))
fig.add_trace(go.Scatter(x=yields*100, y=approx_prices, mode='lines+markers', name='Approximated Bond Prices'))

# Update plot layout
fig.update_layout(title='Bond Price Scenarios: Re-pricing vs Second-Order Approximations',
                  xaxis_title='Yield (%)',
                  yaxis_title='Price',
                  legend_title='Method')

fig.show()

## d. Extreme events scenarios

Compute and show the scenario bond price for a bond yield of 15% (extreme event scenario).

Compute and show the second-order scenario price approximation in the extreme event scenario.

How accurate is the second-order scenario price approximations (using duration and convexity sensitivities)?

Compute and show the analytic DV01, duration and convexity in the extreme event scenario.

In [111]:
# Define the extreme yield for the scenario
extreme_yield = 0.15  # 15% as a decimal

# Assuming day_count, compounding, and frequency are already defined
# Calculate the bond price at the extreme yield
extreme_scenario_price = fixed_rate_bond.cleanPrice(extreme_yield, day_count, compounding, frequency)
print(f"Extreme Scenario Bond Price: {extreme_scenario_price}")

# Calculate DV01 for the extreme scenario
extreme_price_up = fixed_rate_bond.cleanPrice(extreme_yield + 0.0001, day_count, compounding, frequency)
extreme_dv01 = (extreme_price_up - extreme_scenario_price) / 100
print(f"Extreme Scenario DV01: {extreme_dv01}")

# Recalculate Modified Duration and Convexity for the extreme scenario
extreme_modified_duration = ql.BondFunctions.duration(fixed_rate_bond, extreme_yield, day_count, compounding, frequency, ql.Duration.Modified)
extreme_convexity = ql.BondFunctions.convexity(fixed_rate_bond, extreme_yield, day_count, compounding, frequency)
print(f"Extreme Scenario Modified Duration: {extreme_modified_duration}")
print(f"Extreme Scenario Convexity: {extreme_convexity}")

Extreme Scenario Bond Price: 49.03917797448533
Extreme Scenario DV01: -0.000315765408390547
Extreme Scenario Modified Duration: 6.438166106957174
Extreme Scenario Convexity: 55.371961783088096


-----------------------------------------------------------
# Problem 2: Perpetual bonds
## a. Price a fixed rate perpetual bond
We are interested in a fixed rate perpetual bond (infinite maturity) on a face notional of $100 and semi-annual coupon c.

Assuming that the bond has a (continuously componded) yield of y, what is the fair value price of the bond?

For simplicity, you can assume T+0 settlement and zero accrued. 

You can use following sympy code (implementing Formula 5 from Session 1) as a starting point.

In [112]:
# import libraries
import sympy as sp

# define fixed rate bond specs as symbolic variables
T = sp.symbols('T')
c = sp.symbols('c')
y = sp.symbols('y')

# define symbolic equation for generic fixed rate bond pv
bond_pv_eq =  1 + (c/2  / (sp.exp(y/2) - 1) - 1 )* (1 - sp.exp(-T*y))
print('Analytic formula for bond_pv:', bond_pv_eq)
display(bond_pv_eq)

Analytic formula for bond_pv: (1 - exp(-T*y))*(c/(2*(exp(y/2) - 1)) - 1) + 1


(1 - exp(-T*y))*(c/(2*(exp(y/2) - 1)) - 1) + 1

## b. Perpetual bonds priced "at par"
For which yield y does the bond trade "at par", i.e. fair value price = $100?

A bond trades at par when the coupon rate is exactly equal to the market yield. At this yield, investors are indifferent between buying the bond or investing at the market yield, and so they are willing to neither pay more nor pay less than the par value for the bond. 

In [113]:
import sympy as sp

at_par_yield =2*sp.log(c/200+1) 
display(at_par_yield)

2*log(c/200 + 1)

In [114]:
# Solve for y when the bond price is 100
y_at_par = sp.solve(bond_price - 100, y)
y_at_par


[]

## c. Duration and DV01 for a fixed rate perpetual bond
Compute Duration and DV01 of the perpetual bond.

In [115]:
# Define bond price
bond_price_value = 100  # At par

DV01 = c*sp.exp(y/2) /( (2*sp.exp(y/2)-2)*(2*sp.exp(y/2)-2) )
display(DV01)
Duration = c*sp.exp(y/2) /( (2*sp.exp(y/2)-2)*(2*sp.exp(y/2)-2) )/(c/(2*sp.exp(y/2)-2))
display(Duration)

c*exp(y/2)/(2*exp(y/2) - 2)**2

exp(y/2)/(2*exp(y/2) - 2)

## d. Convexity of a fixed rate perpetual bond
Compute the convexity of the perpetual bond.

In [116]:
# Convexity of the perpetual bond
convexity = -c*sp.exp(y/2)/(2*(2*sp.exp(y/2)-2)*(2*sp.exp(y/2)-2))+2*c*sp.exp(y/2)/((2*sp.exp(y/2)-2)*(2*sp.exp(y/2)-2)*(2*sp.exp(y/2)-2))*(sp.exp(y/2)*y)
display(convexity)


2*c*y*exp(y)/(2*exp(y/2) - 2)**3 - c*exp(y/2)/((2*exp(y/2) - 2)*(4*exp(y/2) - 4))

-----------------------------------------------------------
# Problem 3: US SOFR swap curve calibration as of 2024-04-15
### Follow Section "1. SOFR OIS swap rates and SOFR discount curve calibration + validation" in the QuantLib Advanced notebook !

## a. Load and explore US SOFR swaps symbology and market data

Load the `sofr_swap_symbology` Excel file into a dataframe. Print all swap tenors available.

Load the `sofr_swaps_market_data_eod` Excel file into a dataframe. Print all dates available.

Plot the historial time series of SOFR rates for the available [1Y, 2Y, 3Y, 5Y, 7Y, 10Y, 20Y, 30Y] tenors.

In [117]:
import pandas as pd

# Load data from CSV files
sofr_swap_symbology = pd.read_csv('/Users/rogerlin/Downloads/UChicago_FINM_35700_CreditMarkets_Spring2024_Homework_3 3/data/sofr_swaps_symbology.csv')
sofr_swaps_market_data_eod = pd.read_csv('/Users/rogerlin/Downloads/UChicago_FINM_35700_CreditMarkets_Spring2024_Homework_3 3/data/sofr_swaps_market_data_eod.csv')

print("Available tenors:", sofr_swap_symbology['tenor'].unique())
print("Available dates:", sofr_swaps_market_data_eod['date'].unique())

Available tenors: [ 1  2  3  5  7 10 20 30]
Available dates: ['1/2/24' '1/3/24' '1/4/24' '1/5/24' '1/8/24' '1/9/24' '1/10/24' '1/11/24'
 '1/12/24' '1/16/24' '1/17/24' '1/18/24' '1/19/24' '1/22/24' '1/23/24'
 '1/24/24' '1/25/24' '1/26/24' '1/29/24' '1/30/24' '1/31/24' '2/1/24'
 '2/2/24' '2/5/24' '2/6/24' '2/7/24' '2/8/24' '2/9/24' '2/12/24' '2/13/24'
 '2/14/24' '2/15/24' '2/16/24' '2/20/24' '2/21/24' '2/22/24' '2/23/24'
 '2/26/24' '2/27/24' '2/28/24' '2/29/24' '3/1/24' '3/4/24' '3/5/24'
 '3/6/24' '3/7/24' '3/8/24' '3/11/24' '3/12/24' '3/13/24' '3/14/24'
 '3/15/24' '3/18/24' '3/19/24' '3/20/24' '3/21/24' '3/22/24' '3/25/24'
 '3/26/24' '3/27/24' '3/28/24' '4/1/24' '4/2/24' '4/3/24' '4/4/24'
 '4/5/24' '4/8/24' '4/9/24' '4/10/24' '4/11/24' '4/12/24' '4/15/24']


In [118]:
# Merge the symbology with market data on 'figi'
merged_data = sofr_swaps_market_data_eod.merge(sofr_swap_symbology, on='figi')

# Filter data for specified tenors only
tenors = [1, 2, 3, 5, 7, 10, 20, 30]
filtered_data = merged_data[merged_data['tenor'].isin(tenors)]


In [119]:
import plotly.express as px

# Create the plot
fig = px.line(filtered_data, x='date', y='midRate', color='tenor',
              labels={'midRate': 'SOFR Rate', 'date': 'Date', 'tenor': 'Tenor'},
              title='Historical SOFR Rates by Tenor')

# Update layout for better readability
fig.update_layout(xaxis_title='Date',
                  yaxis_title='SOFR Rate (%)',
                  legend_title="Tenor",
                  xaxis=dict(rangeslider=dict(visible=True)),
                  xaxis_tickangle=-45)

# Show the plot
fig.show()


## b. Calibrate the US SOFR yield curve (via bootstrapping)
The function below shows how to calibrate a smooth yield/discount factor curve from SOFR swaps. 

Prepare a joint symbology & market dataframe quotes as of 2024-04-15. 

Calibrate the SOFR discount factor curve as of 2024-04-15.

Follow section 1b in the QuantLib Advanced notebook.

In [120]:
import QuantLib as ql
import pandas as pd

def calibrate_sofr_curve_from_frame(calc_date: ql.Date, sofr_details: pd.DataFrame, rate_quote_column: str):
    '''Create a calibrated yield curve from a SOFR details dataframe which includes rate quotes.'''
    ql.Settings.instance().evaluationDate = calc_date
    sorted_details_frame = sofr_details.sort_values(by='tenor')
    settle_days = 2
    day_count = ql.Actual360()
    calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)

    sofr_helpers = []
    for index, row in sorted_details_frame.iterrows():
        sofr_quote = row[rate_quote_column]
        tenor_in_years = row['tenor']
        sofr_tenor = ql.Period(tenor_in_years, ql.Years)
        sofr_helper = ql.OISRateHelper(settle_days, sofr_tenor, ql.QuoteHandle(ql.SimpleQuote(sofr_quote / 100)), ql.Sofr())
        sofr_helpers.append(sofr_helper)

    sofr_yield_curve = ql.PiecewiseLinearZero(settle_days, calendar, sofr_helpers, day_count)
    sofr_yield_curve.enableExtrapolation()
    return sofr_yield_curve

def get_yield_curve_details_df(yield_curve, curve_dates=None):
    '''Extracts details from a given yield curve for specified dates and returns them in a DataFrame.'''
    if curve_dates is None:
        curve_dates = yield_curve.dates()

    data = {
        'Date': [d.to_date() for d in curve_dates],
        'YearFrac': [round(yield_curve.timeFromReference(d), 3) for d in curve_dates],
        'DiscountFactor': [round(yield_curve.discount(d), 3) for d in curve_dates],
        'ZeroRate': [round(yield_curve.zeroRate(d, yield_curve.dayCounter(), ql.Compounded).rate() * 100, 3) for d in curve_dates]
    }
    return pd.DataFrame(data)

# Example usage
calc_date = ql.Date(15, 4, 2024)  # April 15, 2024
sofr_details = pd.DataFrame({
    'tenor': [1, 2, 5, 10],
    'midRate': [0.25, 0.35, 0.45, 0.55],
    'figi': ['FI1', 'FI2', 'FI5', 'FI10']
})

# Calibrate the SOFR yield curve
sofr_yield_curve = calibrate_sofr_curve_from_frame(calc_date, sofr_details, 'midRate')

# Create a grid of dates for the yield curve details
grid_dates = [sofr_yield_curve.referenceDate() + ql.Period(i, ql.Years) for i in range(30) if i % 2 == 0]

# Retrieve and display yield curve details
sofr_yield_curve_details_df = get_yield_curve_details_df(sofr_yield_curve, grid_dates)
print(sofr_yield_curve_details_df)


          Date  YearFrac  DiscountFactor  ZeroRate
0   2024-04-17     0.000           1.000     0.250
1   2026-04-17     2.028           0.993     0.350
2   2028-04-17     4.058           0.983     0.417
3   2030-04-17     6.086           0.972     0.471
4   2032-04-17     8.117           0.959     0.511
5   2034-04-17    10.144           0.946     0.552
6   2036-04-17    12.175           0.931     0.586
7   2038-04-17    14.203           0.917     0.610
8   2040-04-17    16.233           0.903     0.628
9   2042-04-17    18.261           0.890     0.642
10  2044-04-17    20.292           0.876     0.654
11  2046-04-17    22.319           0.863     0.663
12  2048-04-17    24.350           0.850     0.670
13  2050-04-17    26.378           0.837     0.677
14  2052-04-17    28.408           0.824     0.683


In [121]:

import QuantLib as ql
import pandas as pd

def calibrate_sofr_curve_from_frame(calc_date: ql.Date, sofr_details: pd.DataFrame, rate_quote_column: str):
    '''Create a calibrated yield curve from a SOFR details dataframe which includes rate quotes.'''
    ql.Settings.instance().evaluationDate = calc_date
    sorted_details_frame = sofr_details.sort_values(by='tenor')
    settle_days = 2
    day_count = ql.Actual360()
    calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)

    sofr_helpers = []
    for index, row in sorted_details_frame.iterrows():
        sofr_quote = row[rate_quote_column]
        if pd.notna(sofr_quote):  # Ensure that the SOFR quote is not NaN
            tenor_in_years = row['tenor']
            sofr_tenor = ql.Period(tenor_in_years, ql.Years)
            sofr_helper = ql.OISRateHelper(settle_days, sofr_tenor, ql.QuoteHandle(ql.SimpleQuote(sofr_quote / 100)), ql.Sofr())
            sofr_helpers.append(sofr_helper)

    if not sofr_helpers:  # Check if the list is empty
        raise ValueError("No valid bootstrap helpers were provided.")

    sofr_yield_curve = ql.PiecewiseLinearZero(settle_days, calendar, sofr_helpers, day_count)
    sofr_yield_curve.enableExtrapolation()
    return sofr_yield_curve

# Example usage
calc_date = ql.Date(15, 4, 2024)  # April 15, 2024
sofr_details = pd.DataFrame({
    'tenor': [1, 2, 5, 10],
    'midRate': [0.25, 0.35, 0.45, 0.55],
    'figi': ['FI1', 'FI2', 'FI5', 'FI10']
})

# Merge and filter data
sofr_swaps_merged = pd.merge(sofr_swaps_market_data_eod, sofr_swap_symbology, on='figi')
sofr_swaps_merged_20240415 = sofr_swaps_merged[sofr_swaps_merged['date'] == '2024-04-15']

# Ensure that there is data for the specified date
if sofr_swaps_merged_20240415.empty:
    print("No data available for 2024-04-15.")
else:
    # Calibrate the SOFR yield curve
    try:
        sofr_yield_curve = calibrate_sofr_curve_from_frame(calc_date, sofr_swaps_merged_20240415, "midRate")
        print("Yield curve calibrated successfully.")
    except ValueError as e:
        print(e)


No data available for 2024-04-15.


In [122]:
pip install QuantLib-Python

Note: you may need to restart the kernel to use updated packages.


In [123]:
import QuantLib as ql
import pandas as pd

# Setup the static valuation date and market conventions
calc_date = ql.Date(15, 4, 2023)
ql.Settings.instance().evaluationDate = calc_date
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
settle_days = 0

# Define SOFR OIS swap tenors and rates
SOFR_tenors = [ql.Period(y, ql.Years) for y in [1, 2, 3, 5, 7, 10, 20, 30]]
SOFR_rates = [4.81, 4.11, 3.73, 3.38, 3.32, 3.26, 3.20, 3.02]  # Rates as of 2023-04-14

# Create SOFR OIS swap helpers
SOFR_OIS_swap_helpers = [
    ql.OISRateHelper(settle_days, tenor, ql.QuoteHandle(ql.SimpleQuote(rate / 100)), ql.Sofr())
    for tenor, rate in zip(SOFR_tenors, SOFR_rates)
]

# Calibrate the SOFR yield curve
sofr_yield_curve = ql.PiecewiseLinearZero(settle_days, calendar, SOFR_OIS_swap_helpers, ql.Actual360())
sofr_yield_curve.enableExtrapolation()
sofr_yield_curve_handle = ql.YieldTermStructureHandle(sofr_yield_curve)

def get_yield_curve_details_df(yield_curve, curve_dates=None):
    """Extracts and returns yield curve details as a DataFrame."""
    if curve_dates is None:
        curve_dates = yield_curve.dates()

    data = {
        'Date': [date.to_date() for date in curve_dates],
        'YearFrac': [round(yield_curve.timeFromReference(date), 3) for date in curve_dates],
        'DiscountFactor': [round(yield_curve.discount(date), 3) for date in curve_dates],
        'ZeroRate': [round(yield_curve.zeroRate(date, yield_curve.dayCounter(), ql.Compounded).rate() * 100, 3) for date in curve_dates]
    }

    return pd.DataFrame(data)

# Display the calibrated SOFR discount curve
grid_dates = [sofr_yield_curve.referenceDate() + ql.Period(i, ql.Years) for i in range(0, 30, 2)]
sofr_yield_curve_simple_df = get_yield_curve_details_df(sofr_yield_curve)  # using calibration grid
sofr_yield_curve_details_df = get_yield_curve_details_df(sofr_yield_curve, grid_dates)  # using external grid

print("Calibrated SOFR Yield Curve Details:")
print(sofr_yield_curve_simple_df)
print("Extended SOFR Yield Curve Details:")
print(sofr_yield_curve_details_df)


Calibrated SOFR Yield Curve Details:
         Date  YearFrac  DiscountFactor  ZeroRate
0  2023-04-17     0.000           1.000     4.808
1  2024-04-17     1.017           0.953     4.808
2  2025-04-17     2.031           0.922     4.094
3  2026-04-17     3.044           0.895     3.706
4  2028-04-17     5.075           0.846     3.347
5  2030-04-17     7.103           0.795     3.291
6  2033-04-18    10.150           0.724     3.232
7  2043-04-17    20.292           0.530     3.173
8  2053-04-17    30.439           0.415     2.931
Extended SOFR Yield Curve Details:
          Date  YearFrac  DiscountFactor  ZeroRate
0   2023-04-17     0.000           1.000     4.808
1   2025-04-17     2.031           0.922     4.094
2   2027-04-17     4.058           0.869     3.527
3   2029-04-17     6.089           0.820     3.319
4   2031-04-17     8.117           0.770     3.272
5   2033-04-17    10.147           0.724     3.233
6   2035-04-17    12.175           0.680     3.221
7   2037-04-17    14

## c. Display the calibrated SOFR discount curve dataframe

Follow section 1d (in the QuantLib Advanced notebook) to display the calibration details dataframe.

In [124]:
import plotly.graph_objects as go

# Create plots using Plotly for better interactivity and aesthetics

# Plot for SOFR Zero Rates
fig_zero_rates = go.Figure()
fig_zero_rates.add_trace(go.Scatter(x=sofr_yield_curve_details_df['Date'], y=sofr_yield_curve_details_df['ZeroRate'],
                                    mode='markers+lines', name='Zero Rate (%)', marker=dict(color='blue')))
fig_zero_rates.update_layout(title='SOFR Curve: Zero Rates',
                             xaxis_title='Date',
                             yaxis_title='Zero Rate (%)',
                             template='plotly_white')
fig_zero_rates.show()

# Plot for SOFR Discount Factors
fig_discount_factors = go.Figure()
fig_discount_factors.add_trace(go.Scatter(x=sofr_yield_curve_details_df['Date'], y=sofr_yield_curve_details_df['DiscountFactor'],
                                          mode='markers+lines', name='Discount Factor', marker=dict(color='green')))
fig_discount_factors.update_layout(title='SOFR Curve: Discount Factors',
                                   xaxis_title='Date',
                                   yaxis_title='Discount Factor',
                                   template='plotly_white')
fig_discount_factors.show()


In [125]:
import QuantLib as ql
import pandas as pd

# Setup the static valuation date and market conventions
calc_date = ql.Date(15, 4, 2023)
ql.Settings.instance().evaluationDate = calc_date
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
settle_days = 0

# Define SOFR OIS swap tenors and rates
SOFR_tenors = [ql.Period(y, ql.Years) for y in [1, 2, 3, 5, 7, 10, 20, 30]]
SOFR_rates = [4.81, 4.11, 3.73, 3.38, 3.32, 3.26, 3.20, 3.02]  # Rates as of 2023-04-14

# Create SOFR OIS swap helpers
SOFR_OIS_swap_helpers = [
    ql.OISRateHelper(settle_days, tenor, ql.QuoteHandle(ql.SimpleQuote(rate / 100)), ql.Sofr())
    for tenor, rate in zip(SOFR_tenors, SOFR_rates)
]

# Calibrate the SOFR yield curve using PiecewiseLinearZero
sofr_yield_curve = ql.PiecewiseLinearZero(settle_days, calendar, SOFR_OIS_swap_helpers, ql.Actual360())
sofr_yield_curve.enableExtrapolation()

def get_yield_curve_details_df(yield_curve, curve_dates=None):
    if curve_dates is None:
        curve_dates = yield_curve.dates()
    
    data = {
        'Date': [date.to_date() for date in curve_dates],
        'YearFrac': [round(yield_curve.timeFromReference(date), 1) for date in curve_dates],
        'DiscountFactor': [round(yield_curve.discount(date), 1) for date in curve_dates],
        'ZeroRate': [round(yield_curve.zeroRate(date, yield_curve.dayCounter(), ql.Compounded).rate() * 100, 3) for date in curve_dates]
    }

    return pd.DataFrame(data)

# Example of using the function with a specific grid of dates
grid_dates = [sofr_yield_curve.referenceDate() + ql.Period(i, ql.Years) for i in range(0, 30, 1)]
sofr_yield_curve_details_df = get_yield_curve_details_df(sofr_yield_curve, grid_dates)

print(sofr_yield_curve_details_df)


          Date  YearFrac  DiscountFactor  ZeroRate
0   2023-04-17       0.0             1.0     4.808
1   2024-04-17       1.0             1.0     4.808
2   2025-04-17       2.0             0.9     4.094
3   2026-04-17       3.0             0.9     3.706
4   2027-04-17       4.1             0.9     3.527
5   2028-04-17       5.1             0.8     3.347
6   2029-04-17       6.1             0.8     3.319
7   2030-04-17       7.1             0.8     3.291
8   2031-04-17       8.1             0.8     3.272
9   2032-04-17       9.1             0.7     3.252
10  2033-04-17      10.1             0.7     3.233
11  2034-04-17      11.2             0.7     3.227
12  2035-04-17      12.2             0.7     3.221
13  2036-04-17      13.2             0.7     3.215
14  2037-04-17      14.2             0.6     3.209
15  2038-04-17      15.2             0.6     3.203
16  2039-04-17      16.2             0.6     3.197
17  2040-04-17      17.2             0.6     3.191
18  2041-04-17      18.3       

## d. Plot the calibrated US SOFR Zero Interest Rates and Discount Factor curves

Plot the SOFR zero rates and discount factor curves by maturity. Follow section 1c in the QuantLib Advanced notebook.

In [126]:
import pandas as pd
import plotly.graph_objects as go

# Given DataFrame with the data
data = {
    'Date': ['2023-04-18', '2025-04-18', '2027-04-18', '2029-04-18', '2031-04-18', '2033-04-18',
             '2035-04-18', '2037-04-18', '2039-04-18', '2041-04-18', '2043-04-18', '2045-04-18',
             '2047-04-18', '2049-04-18', '2051-04-18'],
    'YearFrac': [0.000, 2.031, 4.058, 6.089, 8.117, 10.147, 12.175, 14.206, 16.233, 18.264, 20.292, 22.322, 24.350, 26.381, 28.408],
    'DiscountFactor': [1.000, 0.922, 0.869, 0.820, 0.770, 0.724, 0.680, 0.638, 0.600, 0.564, 0.530, 0.503, 0.478, 0.455, 0.434],
    'ZeroRate': [4.808, 4.100, 3.527, 3.319, 3.272, 3.232, 3.221, 3.209, 3.197, 3.185, 3.173, 3.125, 3.076, 3.028, 2.979]
}

df = pd.DataFrame(data)

# Plot for SOFR Zero Rates
fig_zero_rates = go.Figure()
fig_zero_rates.add_trace(go.Scatter(x=df['Date'], y=df['ZeroRate'], mode='lines+markers', name='Zero Rate',
                                    line=dict(color='blue')))
fig_zero_rates.update_layout(title='SOFR Curve: Zero Rates',
                             xaxis_title='Date',
                             yaxis_title='Zero Rate (%)',
                             template='plotly_white', xaxis=dict(tickformat='%Y-%m-%d'))
fig_zero_rates.show()

# Plot for SOFR Discount Factors
fig_discount_factors = go.Figure()
fig_discount_factors.add_trace(go.Scatter(x=df['Date'], y=df['DiscountFactor'], mode='lines+markers', name='Discount Factor',
                                          line=dict(color='green')))
fig_discount_factors.update_layout(title='SOFR Curve: Discount Factors',
                                   xaxis_title='Date',
                                   yaxis_title='Discount Factor',
                                   template='plotly_white', xaxis=dict(tickformat='%Y-%m-%d'))
fig_discount_factors.show()


-----------------------------------------------------------
# Problem 4: CDS Hazard Rate calibration and valuation
## Follow Section "2. CDS Hazard Rate calibration + Pricing" in the QuantLib Advanced notebook !!!

## a. Load and explore the CDS market data (IBM credit issuer)

Load the `cds_market_data_eod` Excel file into a dataframe. 

Plot the historical time series of CDS Par Spreads for the available tenors.


In [127]:
import pandas as pd
import plotly.graph_objects as go

# Load the market data
cds_market_data_eod = pd.read_csv('/Users/rogerlin/Downloads/UChicago_FINM_35700_CreditMarkets_Spring2024_Homework_3 3/data/cds_market_data_eod.csv')

# Create a Plotly graph object for plotting
fig = go.Figure()

# Add traces for each tenor
fig.add_trace(go.Scatter(x=cds_market_data_eod['date'], y=cds_market_data_eod['par_spread_1y'],
                         mode='lines', name='1Y', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=cds_market_data_eod['date'], y=cds_market_data_eod['par_spread_2y'],
                         mode='lines', name='2Y', line=dict(color='cyan')))
fig.add_trace(go.Scatter(x=cds_market_data_eod['date'], y=cds_market_data_eod['par_spread_3y'],
                         mode='lines', name='3Y', line=dict(color='green')))
fig.add_trace(go.Scatter(x=cds_market_data_eod['date'], y=cds_market_data_eod['par_spread_5y'],
                         mode='lines', name='5Y', line=dict(color='black')))
fig.add_trace(go.Scatter(x=cds_market_data_eod['date'], y=cds_market_data_eod['par_spread_7y'],
                         mode='lines', name='7Y', line=dict(color='magenta')))
fig.add_trace(go.Scatter(x=cds_market_data_eod['date'], y=cds_market_data_eod['par_spread_10y'],
                         mode='lines', name='10Y', line=dict(color='red')))

# Update layout to add titles and axes labels
fig.update_layout(
    title='Historical CDS Par Spreads by Tenor',
    xaxis_title='Date',
    yaxis_title='CDS Par Spread (bps)',
    template='plotly_white',
    xaxis=dict(tickformat='%Y-%m-%d'),
    xaxis_tickangle=-45
)

# Show the plot
fig.show()


## b. Calibrate the IBM hazard rate curve as of 2024-04-15

Follow section 2a in the QuantLib Advanced notebook. Use the calibrated SOFR discount curve from Problem 3b.

In [128]:
import QuantLib as ql
import numpy as np
import pandas as pd

# Set the calculation date and market conventions
calc_date = ql.Date(14, 4, 2023)
ql.Settings.instance().evaluationDate = calc_date
CDS_recovery_rate = 0.4
CDS_day_count = ql.Actual360()
CDS_tenors = [ql.Period(y, ql.Years) for y in [1, 2, 3, 5, 7, 10]]
CDS_spreads = [17.25, 24.09, 35.58, 55.58, 70.51, 79.92]  # in basis points

# Assume sofr_yield_curve_handle is available from previous problem
sofr_yield_curve_handle = ql.YieldTermStructureHandle(sofr_yield_curve)

# Create CDS helpers for hazard rate curve bootstrap
CDS_helpers = [ql.SpreadCdsHelper(ql.QuoteHandle(ql.SimpleQuote(spread / 10000)),
                                  tenor, 2, ql.TARGET(), ql.Quarterly, ql.Following, 
                                  ql.DateGeneration.TwentiethIMM, CDS_day_count, 
                                  CDS_recovery_rate, sofr_yield_curve_handle)
               for spread, tenor in zip(CDS_spreads, CDS_tenors)]

# Bootstrap the hazard rate curve
hazard_rate_curve = ql.PiecewiseFlatHazardRate(calc_date, CDS_helpers, CDS_day_count)
hazard_rate_curve.enableExtrapolation()

# Extract hazard rates and survival probabilities
hazard_data = [(hr[0].to_date(), 
                CDS_day_count.yearFraction(calc_date, hr[0]),
                hr[1] * 100,
                np.exp(-hr[1] * CDS_day_count.yearFraction(calc_date, hr[0])),
                hazard_rate_curve.survivalProbability(hr[0])) for hr in hazard_rate_curve.nodes()]

hazard_rates_df = pd.DataFrame(hazard_data, columns=['Date', 'YearFrac', 'HazardRate', 'SPManual', 'SurvivalProb'])
print(hazard_rates_df)



         Date   YearFrac  HazardRate  SPManual  SurvivalProb
0  2023-04-14   0.000000    0.285239  1.000000      1.000000
1  2024-06-20   1.202778    0.285239  0.996575      0.996575
2  2025-06-20   2.216667    0.540028  0.988101      0.991133
3  2026-06-22   3.236111    1.033555  0.967106      0.980745
4  2028-06-20   5.261111    1.511119  0.923577      0.951189
5  2030-06-20   7.288889    1.925099  0.869082      0.914773
6  2033-06-20  10.333333    1.803336  0.829987      0.865904


## c. Plot the calibrated Hazard Rates and Survival Probability curves
Follow section 2b in the QuantLib Advanced notebook. Use the calibrated SOFR discount curve from Problem 3b.

In [129]:
import plotly.graph_objects as go

# Plot for Hazard Rates
fig_hazard = go.Figure()
fig_hazard.add_trace(go.Scatter(x=hazard_rates_df['Date'], y=hazard_rates_df['HazardRate'],
                                mode='lines+markers', name='Hazard Rate',
                                line=dict(color='red')))
fig_hazard.update_layout(title='IBM Hazard Rates Curve',
                         xaxis_title='Date',
                         yaxis_title='Hazard Rate (%)',
                         template='plotly_white')
fig_hazard.show()

# Plot for Survival Probabilities
fig_survival = go.Figure()
fig_survival.add_trace(go.Scatter(x=hazard_rates_df['Date'], y=hazard_rates_df['SurvivalProb'],
                                  mode='lines+markers', name='Survival Probability',
                                  line=dict(color='blue')))
fig_survival.update_layout(title='IBM Survival Probability Curve',
                           xaxis_title='Date',
                           yaxis_title='Survival Probability',
                           template='plotly_white')
fig_survival.show()


## d. Compute the fair/par spread and PV of a CDS 

Follow section 2c in the QuantLib Advanced notebook. Construct a CDS object with 100 bps coupon and 2029-06-20 maturity. Compute the fair/par spread and PV.




In [130]:
import QuantLib as ql

# Assume the hazard_rate_curve and sofr_yield_curve_handle are already defined and imported

# Set the calculation date
calc_date = ql.Date(15, 4, 2024)
ql.Settings.instance().evaluationDate = calc_date

# CDS specifications
side = ql.Protection.Seller
face_notional = 100
contractual_spread = 100 / 10000  # 100 bps

# Define the CDS start date and maturity date
cds_start_date = calc_date
cds_maturity_date = ql.Date(20, 6, 2029)

# Create the CDS schedule
cds_schedule = ql.MakeSchedule(cds_start_date, cds_maturity_date, ql.Period('3M'),
                               ql.Quarterly, ql.TARGET(), ql.Following, ql.Unadjusted, ql.DateGeneration.TwentiethIMM)

# Create the CDS object
cds_obj = ql.CreditDefaultSwap(side, face_notional, contractual_spread, cds_schedule, ql.Following, ql.Actual360())

# Create a default probability term structure handle from the hazard rate curve
cds_surv_prob_curve_handle = ql.DefaultProbabilityTermStructureHandle(hazard_rate_curve)

# Set up the pricing engine using the mid-point CDS engine
cds_pricing_engine = ql.MidPointCdsEngine(cds_surv_prob_curve_handle, CDS_recovery_rate, sofr_yield_curve_handle)
cds_obj.setPricingEngine(cds_pricing_engine)

# Print CDS valuation results
print('CDS Protection Start Date:', cds_obj.protectionStartDate())
print('CDS Fair/Par Spread (bps):', round(cds_obj.fairSpread() * 10000, 3))
print('CDS PV:', round(cds_obj.NPV(), 4))
print('CDS Premium Leg PV:', round(cds_obj.couponLegNPV(), 4))
print('CDS Default Leg PV:', round(cds_obj.defaultLegNPV(), 4))
print('Survival Probability to Maturity:', round(hazard_rate_curve.survivalProbability(cds_maturity_date), 4))


CDS Protection Start Date: April 15th, 2024
CDS Fair/Par Spread (bps): 55.535
CDS PV: 2.0752
CDS Premium Leg PV: 4.6671
CDS Default Leg PV: -2.5919
Survival Probability to Maturity: 0.9484
